In [2]:
%load_ext autoreload
%autoreload 2
    
import os, sys
#PWD = os.getenv('PWD')
PWD = '/Users/chris/Documents/lab/emAnalysis/backend/server'

PROJ_MISSING_MSG = """Set an enviroment variable:\n
`DJANGO_PROJECT=your_project_name`\n
or call:\n
`init_django(your_project_name)`
"""

def init_django(project_name=None):
    os.chdir(PWD)
    project_name = project_name or os.environ.get('DJANGO_PROJECT') or None
    if project_name == None:
        raise Exception(PROJ_MISSING_MSG)
    sys.path.insert(0, os.getenv('PWD'))
    os.environ.setdefault('DJANGO_SETTINGS_MODULE', f'{project_name}.settings')
    os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
    import django
    django.setup()

init_django("server")
from django.conf import settings
media_root = settings.MEDIA_ROOT
import numpy as np
import os
from matplotlib import pyplot as plt
import functools
import math
import sims
import copy
import os
from PIL import Image
from pathlib import Path
import numpy as np
from skimage.measure import regionprops
from scipy import spatial
import pickle
from matplotlib.figure import Figure
import pyvips
from image.models import Image
from core.models import Canvas
from mims.models import MIMSImage, MIMSImageSet, MIMSAlignment
from PIL import Image
from scipy.ndimage import rotate
import pprint
import sims
import cv2

In [15]:
import SimpleITK as sitk

from mims.services.registration_utils import create_composite_mask
def command_iteration(method):
    """ Callback invoked when the optimization has an iteration """
    print(f"{method.GetOptimizerIteration():3} " + f"= {method.GetMetricValue():10.5f}")

imageset = MIMSImageSet.objects.get(pk="f45f95ee-aae5-4dc3-b467-77167ecb2462")
mims_images = MIMSImage.objects.filter(image_set=imageset)
mims_image = [m for m in mims_images if "1035_position_2" in m.file.name][0]
reg_loc = os.path.join(media_root, mims_image.file.path[:-3], "registration")


# Read the fixed and moving images
fixed_image = sitk.ReadImage(os.path.join(reg_loc, "em_reg_mask.tiff"), sitk.sitkFloat32)
moving_image = sitk.ReadImage(os.path.join(reg_loc, "mims_reg_mask.tiff"), sitk.sitkFloat32)

# Set up the BSpline transformation
transformDomainMeshSize = [8] * moving_image.GetDimension()
tx = sitk.BSplineTransformInitializer(fixed_image, transformDomainMeshSize)

# Set up the image registration method
registration = sitk.ImageRegistrationMethod()
registration.SetMetricAsMeanSquares()
registration.SetOptimizerAsLBFGSB(
    gradientConvergenceTolerance=1e-5,
    numberOfIterations=100,
    maximumNumberOfCorrections=5,
    maximumNumberOfFunctionEvaluations=1000,
    costFunctionConvergenceFactor=1e7,
)
registration.SetInitialTransform(tx, True)
registration.SetInterpolator(sitk.sitkLinear)

registration.AddCommand(sitk.sitkIterationEvent, lambda: command_iteration(registration))

outTx = registration.Execute(fixed_image, moving_image)
print("-------")
print(outTx)
print(f"Optimizer stop condition: {registration.GetOptimizerStopConditionDescription()}")
print(f" Iteration: {registration.GetOptimizerIteration()}")
print(f" Metric value: {registration.GetMetricValue()}")

sitk.WriteTransform(outTx, os.path.join(reg_loc, "mims_transform.tfm"))

resampler = sitk.ResampleImageFilter()
resampler.SetReferenceImage(fixed_image)
resampler.SetInterpolator(sitk.sitkLinear)
resampler.SetDefaultPixelValue(100)
resampler.SetTransform(outTx)

out = resampler.Execute(moving_image)
simg1 = sitk.Cast(sitk.RescaleIntensity(fixed_image), sitk.sitkUInt8)
simg2 = sitk.Cast(sitk.RescaleIntensity(out), sitk.sitkUInt8)
cimg = sitk.Compose(simg1, simg2, simg1 // 2.0 + simg2 // 2.0)
sitk.WriteImage(simg2, os.path.join(reg_loc, 'mims_mask_unwarped.tif'))
unwarped_composite = create_composite_mask(simg1, simg2)
sitk.WriteImage(cimg, 'mims_mask_overlay.tif')
return_images = {"fixed": fixed_image, "moving": moving_image, "composition": cimg}




  0 =  899.90617
  0 =  897.19564
  0 =  891.25012
  0 =  869.71683
  0 =  787.99713
  0 =  613.48435
  0 =  613.48435
  1 =  448.65687
  1 =  448.65687
  2 =  342.92949
  2 =  342.92949
  3 =  286.94434
  3 =  286.94434
  4 =  248.41900
  4 =  248.41900
  5 =  234.75450
  5 =  234.75450
  6 =  224.51999
  6 =  224.51999
  7 =  213.72249
  7 =  213.72249
  8 =  199.90347
  8 =  199.90347
  9 =  190.59088
  9 =  190.59088
 10 =  184.55328
 10 =  184.55328
 11 =  181.65371
 11 =  181.65371
 12 =  177.50137
 12 =  177.50137
 13 =  171.45478
 13 =  171.45478
 14 =  164.11537
 14 =  164.11537
 15 =  157.34540
 15 =  157.34540
 16 =  152.48330
 16 =  152.48330
 17 =  148.03792
 17 =  148.03792
 18 =  145.33869
 18 =  145.33869
 19 =  142.10280
 19 =  142.10280
 20 =  141.17855
 20 =  141.17855
 21 =  138.38069
 21 =  138.38069
 22 =  137.65387
 22 =  137.65387
 23 =  135.14926
 23 =  135.14926
 24 =  134.47391
 24 =  134.47391
 25 =  133.72638
 25 =  133.72638
 26 =  132.88108
 26 =  132.881